# Diabetes Readmission prediction using Artificial Neural Network (MultiLayerPerceptron) model

In [1]:
# Data processing
from pyspark.sql.functions import log, col, exp

# Modeling
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [0]:
write_path = 'dbfs:/tmp/reproducible_ml_uofl/diab_readmit_csv.delta'
diabetes_readmit = spark.read.format('delta').load(write_path)

#Show basic summary stats
display(diabetes_readmit.summary())

summary,patient_nbr,time_in_hospital,num_procedures,num_lab_procedures,num_medications,number_outpatient,number_inpatient,number_emergency,number_diagnoses,gender_cd,DiabetesMedication,readmit_flag,race_cd
count,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766
mean,5.4330400694947235E7,4.395986871843248,1.339730361810428,43.09564098028811,16.021844230882614,0.36935715268360747,0.635565906098304,0.19783621248747127,7.422606764538254,0.5375862272271682,0.7700312481575379,0.11159915885462728,null
stddev,3.869635934653421E7,2.985107767471267,1.705806979121172,19.674362249142096,8.127566209167309,1.2672650965326817,1.26286329009732,0.9304722684224632,1.9336001449974247,0.49858772375671534,0.420814525814695,0.3148741984505526,null
min,135,1,0,1,1,0,0,0,1,0,0,0,AfrAmr
25%,23412645,2,0,31,10,0,0,0,6,0,1,0,null
50%,45500490,4,1,44,15,0,0,0,8,1,1,0,null
75%,87532902,6,2,57,20,0,1,0,9,1,1,0,null
max,189502619,14,6,132,81,42,21,76,16,1,1,1,White


In [0]:
# Train test split
trainDF, testDF = diabetes_readmit.randomSplit([.65, .35], seed=42)
# Print the number of records
print(f'There are {trainDF.cache().count()} records in the training dataset.')
print(f'There are {testDF.cache().count()} records in the testing dataset.')

There are 66284 records in the training dataset.
There are 35482 records in the testing dataset.

##Now we need to modify the categorical variable race_cd into one-hot-encoded version
 For this we can either use the StringIndexer and OneHotEncoder separately OR use a pipeline to do this in one step

In [0]:
#You can also create a pipeline and do everything together in one easy fit and transform step
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
 
categoricalColumns = ["race_cd"]
stages = [] # stages in Pipeline
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    
# Use OneHotEncoder to convert categorical variables into binary SparseVectors
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])

# Define the pipeline based on the stages created in previous steps.
pipeline = Pipeline(stages=[stringIndexer, encoder])
 
# Define the pipeline model.
transform_mdl = pipeline.fit(trainDF)
trainDF21=transform_mdl.transform(trainDF)
trainDF21.show()

+-----------+----------------+--------------+------------------+---------------+-----------------+----------------+----------------+----------------+---------+------------------+------------+-------+------------+---------------+
patient_nbr|time_in_hospital|num_procedures|num_lab_procedures|num_medications|number_outpatient|number_inpatient|number_emergency|number_diagnoses|gender_cd|DiabetesMedication|readmit_flag|race_cd|race_cdIndex|race_cdclassVec|
+-----------+----------------+--------------+------------------+---------------+-----------------+----------------+----------------+----------------+---------+------------------+------------+-------+------------+---------------+
 135| 3| 1| 31| 14| 0| 1| 0| 5| 1| 1| 0| White| 0.0| (4,[0],[1.0])|
 135| 8| 6| 77| 33| 0| 0| 0| 8| 1| 1| 1| White| 0.0| (4,[0],[1.0])|
 729| 4| 2| 68| 23| 0| 0| 0| 9| 1| 1| 0| White| 0.0| (4,[0],[1.0])|
 927| 5| 0| 49| 5| 0| 0| 0| 3| 1| 1| 0| AfrAmr| 1.0| (4,[1],[1.0])|
 1152| 6| 4| 45| 15| 0| 2| 0| 6| 1| 1| 0| AfrAmr| 1.0| (4,[1],[1.0])|
 1152| 12| 1| 37| 18| 0| 2| 0| 5| 1| 1| 0| AfrAmr| 1.0| (4,[1],[1.0])|
 1314| 1| 5| 47| 12| 0| 1| 0| 7| 1| 1| 1| White| 0.0| (4,[0],[1.0])|
 2025| 12| 2| 47| 18| 0| 0| 0| 9| 1| 1| 0| White| 0.0| (4,[0],[1.0])|
 3069| 8| 6| 57| 31| 0| 0| 0| 9| 0| 1| 0| White| 0.0| (4,[0],[1.0])|
 4302| 1| 1| 31| 9| 0| 0| 0| 7| 1| 0| 0| White| 0.0| (4,[0],[1.0])|
 5220| 2| 0| 15| 14| 0| 0| 0| 9| 0| 1| 0| White| 0.0| (4,[0],[1.0])|
 5220| 11| 4| 65| 19| 0| 2| 0| 9| 0| 1| 0| White| 0.0| (4,[0],[1.0])|
 6174| 7| 0| 77| 12| 0| 0| 0| 9| 1| 1| 0| White| 0.0| (4,[0],[1.0])|
 6228| 4| 4| 47| 16| 0| 0| 0| 7| 1| 0| 0| White| 0.0| (4,[0],[1.0])|
 6309| 1| 5| 35| 13| 0| 0| 0| 8| 0| 1| 0| White| 0.0| (4,[0],[1.0])|
 6939| 6| 2| 65| 19| 0| 0| 0| 7| 0| 1| 0| White| 0.0| (4,[0],[1.0])|
 8154| 1| 0| 58| 5| 0| 0| 0| 8| 1| 0| 0| White| 0.0| (4,[0],[1.0])|
 8748| 2| 0| 64| 10| 0| 0| 0| 9| 0| 1| 0| AfrAmr| 1.0| (4,[1],[1.0])|
 10377| 1| 2| 33| 12| 0| 0| 0| 5| 0| 1| 0| White| 0.0| (4,[0],[1.0])|
 10422| 4| 2| 54| 3| 0| 0| 0| 8| 1| 0| 0| AfrAmr| 1.0| (4,[1],[1.0])|
+-----------+----------------+--------------+------------------+---------------+-----------------+----------------+----------------+----------------+---------+------------------+------------+-------+------------+---------------+
only showing top 20 rows

In [0]:
# Linear regression expect a vector input
vecAssembler = VectorAssembler(inputCols=['time_in_hospital','num_procedures','num_medications', 'number_inpatient','number_emergency','number_diagnoses','DiabetesMedication'], outputCol="features")
vecTrainDF = vecAssembler.transform(trainDF21)

In [0]:
# input layer of size 7 (features), one intermediate of size 10, and output of size 2 (classes)
layers = [7,10,2]

# Create Decision tree calssifier
ann_mlpc = (MultilayerPerceptronClassifier(labelCol='readmit_flag',
                                            featuresCol='features',
                                            maxIter=100,
                                            layers=layers,
                                            blockSize=128,
                                            seed=1234))
ann_mdl = ann_mlpc.fit(vecTrainDF)
predict_train = ann_mdl.transform(vecTrainDF)
predict_train.select('readmit_flag', 'rawPrediction', 'prediction', 'probability').show(10)


+------------+--------------------+----------+--------------------+
readmit_flag| rawPrediction|prediction| probability|
+------------+--------------------+----------+--------------------+
 0|[1.62837038739449...| 0.0|[0.88771755036096...|
 1|[1.63238044136749...| 0.0|[0.88740691909947...|
 0|[1.69225982577089...| 0.0|[0.90483580832412...|
 0|[1.64457477068628...| 0.0|[0.92306362107251...|
 0|[1.36696366292519...| 0.0|[0.81017714479284...|
 0|[1.43063105347940...| 0.0|[0.85425130105974...|
 1|[1.78511982336359...| 0.0|[0.91037017168271...|
 0|[1.57047752770136...| 0.0|[0.89510268928386...|
 0|[1.59426559402237...| 0.0|[0.88095238421950...|
 0|[1.93066411360776...| 0.0|[0.94318175921958...|
+------------+--------------------+----------+--------------------+
only showing top 10 rows

In [0]:
testDF21=transform_mdl.transform(testDF) #do the data transformation using saved parameters from training
vecTestDF = vecAssembler.transform(testDF21) #do the feature transformation using vector assembler
# Make predictions on testing dataset
predict_test = ann_mdl.transform(vecTestDF) #make predictions using the trained model

In [0]:
eval_ann = BinaryClassificationEvaluator(labelCol = "readmit_flag")
auc_train = eval_ann.evaluate(predict_train)
print(auc_train)

auc_test = eval_ann.evaluate(predict_test)
print(auc_test)

0.634270504850411
0.6345223971936924

In [0]:
import pandas as pd
import time 
import sys
from pyspark.sql.window import *
from pyspark.sql.types import * 
from pyspark.sql.functions import *
import pyspark.sql.functions as f

def get_model_stats(print_lable,dataset,evaluator,mbr_id,target):
  print(print_lable)
  AUROC=evaluator.evaluate(dataset)
  
  tot_event=dataset.count()
  tot_target=dataset.select(target).filter(f.col(target)==1).count()
  inc_rate=float(tot_target)/float(tot_event)
  print('Incidence rate in dataset: ',inc_rate)
  print('AUROC: ',AUROC)
  

  def mbr_prob1(prob_vec):
    return float(prob_vec[1]) #Get the second value from probability vector of the prediction since that is probability of target 1
  def mbr_prob2(rawp_vec):
    prob_vec = np.exp(rawp_vector)/(1+ np.exp(rawp_vec)) #this is the same as the logit function e**x/1+e**x
    return float(prob_vec[1])
  get_prob_1= udf(mbr_prob1, FloatType())
  get_prob_2= udf(mbr_prob2, FloatType())

  try:
    df_plot_tmp = dataset.withColumn('prob_score', get_prob_1('probability'))
  except:
    df_plot_tmp = dataset.withColumn('prob_score', get_prob_2('rawPrediction'))
  df_reg1=(df_plot_tmp.select(mbr_id,"rawPrediction","probability","prob_score",target))

  #Create Percentiles based on the predicted probability
  df_reg_12=  df_reg1.sort(col("prob_score").desc())
  df_reg_12 = df_reg_12.withColumn("new_column",lit("ABC"))
  w = Window().partitionBy("new_column").orderBy(col("prob_score").desc())  
  df_reg_12 = df_reg_12.withColumn("row_num",row_number().over(w)).drop("new_column")
  df_reg2 = (df_reg_12
             .withColumn("score_pctl", f.percent_rank().over(Window.orderBy(f.col("row_num"))))
             .withColumn("pctl_cat", f.when(f.col("score_pctl")==1, f.lit(100)).otherwise(f.floor(f.col("score_pctl")*100) + 1))
             )
  df_reg22 = (df_reg2
              .groupBy("pctl_cat")
              .agg(f.mean(target).alias("pctl_target_rate"),\
                   (f.sum(target)).alias("pctl_target_total"),\
                   (f.count(mbr_id)).alias("total_count"),\
                   (f.mean("prob_score")).alias("mean_predicted_prob")
                  ,)
              .orderBy(f.col("pctl_cat"))
              )

  #Now let's create the cumulative percentiles
  cum_sum = (df_reg22
              .withColumn('cum_target', f.sum(df_reg22.pctl_target_total).over(Window.partitionBy().orderBy().rowsBetween(-sys.maxsize, 0)))
              .withColumn('cum_total', f.sum(df_reg22.total_count).over(Window.partitionBy().orderBy().rowsBetween(-sys.maxsize, 0)))
              .withColumn('cum_target_cap_rate',f.col("cum_target")/f.col("cum_total"))
              .withColumn('cum_target_cap_of_total',((f.col("cum_target")/tot_target))*100)
            )
  return cum_sum

In [0]:
cum_cap_dt = get_model_stats("Artificial Neural Network",predict_test,eval_ann,"patient_nbr","readmit_flag")

Artificial Neural Network
Incidence rate in dataset: 0.11225410067076264
AUROC: 0.6345223971936924

In [0]:
display(cum_cap_dt)

pctl_cat,pctl_target_rate,pctl_target_total,total_count,mean_predicted_prob,cum_target,cum_total,cum_target_cap_rate,cum_target_cap_of_total
1,0.4253521126760563,151,355,0.3884231478395596,151,355,0.4253521126760563,3.7911122269646
2,0.29577464788732394,105,355,0.32032184172684036,256,710,0.36056338028169016,6.427316093396937
3,0.2704225352112676,96,355,0.28197051102006937,352,1065,0.3305164319248826,8.837559628420788
4,0.22535211267605634,80,355,0.2534388258423604,432,1420,0.30422535211267604,10.846095907607332
5,0.2028169014084507,72,355,0.23392344494940528,504,1775,0.28394366197183096,12.65377855887522
6,0.192090395480226,68,354,0.22065006550085747,572,2129,0.2686707374354157,14.361034396183781
7,0.19718309859154928,70,355,0.2064826678222334,642,2484,0.2584541062801932,16.118503640472007
8,0.2084507042253521,74,355,0.19442139511377038,716,2839,0.2522014793941529,17.976399698719558
9,0.19436619718309858,69,355,0.18582005047462355,785,3194,0.24577332498434565,19.708762239517952
10,0.2028169014084507,72,355,0.17912935396315346,857,3549,0.241476472245703,21.51644489078584
